In [ ]:
%run ../config/load_config

In [ ]:
# Create a VIEW for Arrival Performance in the Gold layer
print(f"Creating VIEW {catalog}.{schema_gold}.arrival_performance_kpi...", end="")

spark.sql(f"""
CREATE OR REPLACE VIEW {catalog}.{schema_gold}.arrival_performance_kpi AS
SELECT
    al.arrival_event_id AS event_id,
    al.line_id,
    al.naptan_id,
    al.station_name,
    al.expected_arrival,
    al.time_to_live,
    al.event_timestamp,
    ROUND((unix_timestamp(al.event_timestamp) - unix_timestamp(al.expected_arrival))/60.0,2) 
        AS arrival_delay_minutes,
    CASE
        WHEN al.time_to_station <= 120 THEN 'good'
        WHEN al.time_to_station <= 300 THEN 'acceptable'
        ELSE 'poor'
    END AS regularity_status,
    ls.is_service_disrupted,
    ls.disruption_category,
    ls.disruption_description,
    sg.borough_name AS borough
FROM {catalog}.{schema_silver}.arrival_events_sv al
LEFT JOIN {catalog}.{schema_silver}.stops_geo_sv sg ON al.naptan_id = sg.naptan_id
LEFT JOIN {catalog}.{schema_silver}.lines_sv ls ON ls.line_id = al.line_id
"""
)
print("done")